In [3]:
import numpy as np
import pandas as pd

def determine_lifetime(df, threshold=0.5):
    """
    Determine the lifetime of LEDs based on their luminosity decay.
    
    Parameters:
        df (pd.DataFrame): DataFrame with time and test columns representing luminosity.
        threshold (float): The luminosity threshold at which the LED is considered dead.
    
    Returns:
        tuple: A dictionary of lifetimes and a list indicating if the lifetime was prolonged (1 for prolonged, 0 otherwise).
    """
    time_col = df['time']
    lifetimes = {}
    prolonged = []

    for col in df.columns:
        if col == 'time':
            continue
        
        # Find the first time the luminosity drops below the threshold
        luminosity = df[col]
        below_threshold = luminosity < threshold
        
        if below_threshold.any():
            # Get the time at which the threshold is crossed
            death_index = below_threshold.idxmax()  # First occurrence of True
            lifetimes[col] = time_col.iloc[death_index]
            prolonged.append(0)  # Not prolonged
        else:
            # Approximate using exponential decay
            # Fit an exponential function: luminosity = a * exp(-b * time)
            log_luminosity = np.log(luminosity)
            coeffs = np.polyfit(time_col, log_luminosity, 1)  # Fit log-linear model
            b, log_a = -coeffs[0], coeffs[1]
            a = np.exp(log_a)
            
            # Solve for time when luminosity = threshold
            death_time = np.log(threshold / a) / -b
            lifetimes[col] = death_time
            prolonged.append(1)  # Prolonged
    
    return lifetimes, prolonged


# Load the dataset
df = pd.read_csv("S1.csv")

# Calculate lifetimes and prolongation status for the given DataFrame
led_lifetimes, prolongation_status = determine_lifetime(df)

# Display results
print("Lifetimes:", led_lifetimes)
print("Prolongation Status:", prolongation_status)


{'test_0': 4943.802479576001, 'test_1': 6122.724239045018, 'test_2': 6274.78203524984, 'test_3': 1392, 'test_4': 5930.1665733777145, 'test_5': 6301.083145353688, 'test_6': 5349.0560593848795, 'test_7': 1128, 'test_8': 5317.426150752668, 'test_9': 4988.045022705996, 'test_10': 1128, 'test_11': 4267.506949723805, 'test_12': 960, 'test_13': 3992.045652499671, 'test_14': 6375.104572570105}
